In [1]:
import pandas as pd
import requests
import os

df = pd.read_csv('cs552j_A1_dataset_image_id_url.csv')

In [6]:
# Folder where you want to save the image
save_folder = "a1_images"
os.makedirs(save_folder, exist_ok=True)

In [8]:
def download_image(image_url, save_folder):
    image_name = os.path.join(save_folder, image_url.split("/")[-1])
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(image_name, "wb") as file:
            file.write(response.content)
        print(f"Image saved at: {image_name}")
    else:
        print("Failed to download image")

In [12]:
# # downloads all the images to your computer. Do this only once locally, then you can continue.

for i in df.coco_url.values:
    download_image(i, save_folder)

# Your Assessment code below

- please make good use of markdown to separate the sections. 

In [20]:
train_path = os.path.join('Datasets', "Training_Data")
test_path = os.path.join('Datasets', "Testing_Data")
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

In [14]:
#train test split
from sklearn.model_selection import train_test_split
import shutil

In [5]:
print(df["label"].value_counts())

label
walking_running    98
sitting            95
standing           92
Name: count, dtype: int64


In [12]:
df.head(3)

,Unnamed: 0,license,file_name,coco_url,height,width,date_captured,flickr_url,id,label
0,2,4,000000252219.jpg,http://images.cocodataset.org/val2017/00000025...,428,640,2013-11-14 22:32:02,http://farm4.staticflickr.com/3446/3232237447_...,252219,walking_running
1,11,1,000000386912.jpg,http://images.cocodataset.org/val2017/00000038...,480,640,2013-11-15 16:38:19,http://farm5.staticflickr.com/4088/4980393979_...,386912,sitting
2,27,3,000000308394.jpg,http://images.cocodataset.org/val2017/00000030...,428,640,2013-11-17 03:48:10,http://farm4.staticflickr.com/3152/2818802025_...,308394,sitting


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

In [22]:


# Function to copy images to the destination folder
def save_images(image_list, destination_folder):
    for _, row in image_list.iterrows():
        src_path = os.path.join('images', row["file_name"])  # Source path
        # print(src_path)
        dst_path = os.path.join(destination_folder, row["file_name"])  # Destination path
        if os.path.exists(src_path):  # Ensure the file exists
            shutil.copy(src_path, dst_path)

# Save images
save_images(train_df, train_path)
save_images(test_df, test_path)